In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import random
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import GradScaler, autocast

import cv2

import albumentations
from albumentations.pytorch import ToTensorV2

from sklearn.metrics import confusion_matrix, roc_auc_score, average_precision_score

from tqdm import tqdm
import argparse
import os, sys, yaml

sys.path.append('/workspace/siim-rsna-2021')
from src.logger import setup_logger, LOGGER
from src.meter import mAPMeter, AUCMeter, APMeter, AverageValueMeter
from src.utils import plot_sample_images
from src.segloss import SymmetricLovaszLoss


# import neptune.new as neptune
import wandb
import pydicom

import time
from contextlib import contextmanager

import timm

import warnings

target_columns = [
    "Negative for Pneumonia", "Typical Appearance", "Indeterminate Appearance", "Atypical Appearance", "is_none"
]


@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f'[{name}] done in {time.time() - t0:.0f} s')

In [2]:
from exp.exp414.train import Net as Net414
from exp.exp416.train import Net as Net416

from exp.exp418.train import Net as Net418
from exp.exp419.train import Net as Net419
from exp.exp420.train import Net as Net420

from exp.exp520.train import Net as Net520
from exp.exp551.train import Net as Net551
from exp.exp552.train import Net as Net552
from exp.exp553.train import Net as Net553

from exp.exp605.train import Net as Net605

In [3]:
class CustomDataset(Dataset):
    def __init__(self,
                 df,
                 image_size=512,
                 transform=None,
                 ):
        self.df = df
        self.image_size = image_size
        self.transform = transform
        self.cols = target_columns

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        row = self.df.iloc[index]
        images = cv2.imread(row.npy_path)

        # original image size
        original_h = images.shape[0]
        original_w = images.shape[1]
        images = cv2.resize(images, (512, 512))

        if self.transform is not None:
            aug = self.transform(image=images)
            images_only = aug['image'].astype(np.float32).transpose(2, 0, 1) / 255
        return {
            "image": torch.tensor(images_only, dtype=torch.float),
        }


In [4]:
def get_val_transforms(image_size=512):
    return albumentations.Compose([
        albumentations.Resize(image_size, image_size),
])

In [5]:
study_sub_list = [

#     # prediction set, b6 mask
#     [
#         # backbone
#         Net414("tf_efficientnetv2_m_in21k"),
#         "tf_efficientnetv2_m_in21k",
#         # img_size
#         512,
#         # weight list
#         [
#             "/workspace/output/exp414/model/cv0_weight_checkpoint_best.pth",
#             "/workspace/output/exp414/model/cv1_weight_checkpoint_best.pth",
#             "/workspace/output/exp414/model/cv2_weight_checkpoint_best.pth",
#             "/workspace/output/exp414/model/cv3_weight_checkpoint_best.pth",
#             "/workspace/output/exp414/model/cv4_weight_checkpoint_best.pth",
#         ],
#     ],

#     # prediction set, b7 mask
#     [
#         # backbone
#         Net416("tf_efficientnetv2_m_in21k"),
#         "tf_efficientnetv2_m_in21k",
#         # img_size
#         512,
#         # weight list
#         [
#             "/workspace/output/exp416/model/cv0_weight_checkpoint_best.pth",
#             "/workspace/output/exp416/model/cv1_weight_checkpoint_best.pth",
#             "/workspace/output/exp416/model/cv2_weight_checkpoint_best.pth",
#             "/workspace/output/exp416/model/cv3_weight_checkpoint_best.pth",
#             "/workspace/output/exp416/model/cv4_weight_checkpoint_best.pth",
#         ],
#     ],

    # ===========================================
    # Eff v2 L
    # ===========================================


    # prediction set, b7 map
    [
        # backbone
        Net419("tf_efficientnetv2_l_in21k"),
        "tf_efficientnetv2_l_in21k",
        # img_size
        512,
        # weight list
        [
            "/workspace/output/exp419/model/cv0_weight_checkpoint_best.pth",
            "/workspace/output/exp419/model/cv1_weight_checkpoint_best.pth",
            "/workspace/output/exp419/model/cv2_weight_checkpoint_best.pth",
            "/workspace/output/exp419/model/cv3_weight_checkpoint_best.pth",
            "/workspace/output/exp419/model/cv4_weight_checkpoint_best.pth",
        ],
    ],

    # prediction set, b6, b7
    [
        # backbone
        Net420("tf_efficientnetv2_l_in21k"),
        "tf_efficientnetv2_l_in21k",
        # img_size
        512,
        # weight list
        [
            "/workspace/output/exp420/model/cv0_weight_checkpoint_best.pth",
            "/workspace/output/exp420/model/cv1_weight_checkpoint_best.pth",
            "/workspace/output/exp420/model/cv2_weight_checkpoint_best.pth",
            "/workspace/output/exp420/model/cv3_weight_checkpoint_best.pth",
            "/workspace/output/exp420/model/cv4_weight_checkpoint_best.pth",
        ],
    ],
    
    # prediction set, b6, b7
    [
        # backbone
        Net420("tf_efficientnetv2_l_in21k"),
        "tf_efficientnetv2_l_in21k",
        # img_size
        512,
        # weight list
        [
            "/workspace/output/exp520/model/cv0_weight_checkpoint_best.pth",
            "/workspace/output/exp520/model/cv1_weight_checkpoint_best.pth",
            "/workspace/output/exp520/model/cv2_weight_checkpoint_best.pth",
            "/workspace/output/exp520/model/cv3_weight_checkpoint_best.pth",
            "/workspace/output/exp520/model/cv4_weight_checkpoint_best.pth",
        ],
    ],
    
    # prediction set, b6
    [
        # backbone
        Net418("tf_efficientnetv2_l_in21k"),
        "tf_efficientnetv2_l_in21k",
        # img_size
        512,
        # weight list
        [
            "/workspace/output/exp551/model/cv0_weight_checkpoint_best.pth",
            "/workspace/output/exp551/model/cv1_weight_checkpoint_best.pth",
            "/workspace/output/exp551/model/cv2_weight_checkpoint_best.pth",
            "/workspace/output/exp551/model/cv3_weight_checkpoint_best.pth",
            "/workspace/output/exp551/model/cv4_weight_checkpoint_best.pth",
        ],
    ],

    # ===========================================
    # Swin transformer
    # ===========================================

#     # prediction set
#     [
#         # backbone
#         Net605("swin_base_patch4_window12_384"),
#         "swin_base_patch4_window12_384",
#         # img_size
#         384,
#         # weight list
#         [
#             "/workspace/output/exp605/model/cv0_weight_checkpoint_best.pth",
#             "/workspace/output/exp605/model/cv1_weight_checkpoint_best.pth",
#             "/workspace/output/exp605/model/cv2_weight_checkpoint_best.pth",
#             "/workspace/output/exp605/model/cv3_weight_checkpoint_best.pth",
#             "/workspace/output/exp605/model/cv4_weight_checkpoint_best.pth",
#         ],
#     ],
    
    

]

In [6]:
device = "cuda:0"

In [7]:
from copy import deepcopy

# key: image size, value: model
model_dict = {}

image_size_list = []

for model_set in tqdm(study_sub_list):
    # 画像サイズをkey, modelのリストをvalueにする
    # keyがまだない場合はからのリストを登録
    model_dict.setdefault(model_set[2], [])
    model_list = []
    for ckpt in tqdm(model_set[3]):
        model = model_set[0].to(device)
        weight = torch.load(ckpt, map_location=device)
        model.load_state_dict(weight["state_dict"])
        model.eval()
        model_list.append(deepcopy(model))
    model_dict[model_set[2]] += model_list
    image_size_list.append(model_set[2])
    
image_size_list = list(set(image_size_list))

100%|██████████| 4/4 [00:12<00:00,  3.05s/it]


In [8]:
df_ext = pd.read_csv('/workspace/data/external/df_ext.csv')

In [9]:
dataset = CustomDataset(df=df_ext, transform=get_val_transforms(512))
test_loader = DataLoader(
    dataset,
    shuffle=False,
    batch_size=32,
    num_workers=12,
    pin_memory=True,
)

In [10]:
pred_list1 = []
pred_mask1 = []

for i, image in tqdm(enumerate(test_loader)):
#     if i == 3:
#         break
    pred_list2 = []
    pred_mask2 = []
#     image = dataset[i]["image"].to(device).unsqueeze(0)
    image = image["image"].to(device)
    with torch.no_grad():
        for img_size, model_list in model_dict.items():
            for model in model_list:
                preds, *mask = model(image)
                preds = preds.cpu().detach()
                
                pred_list2.append(preds.sigmoid())
                pred_mask2 += [m.cpu().detach().sigmoid() for m in mask]
                
        # average prediction
        pred_list1.append(torch.stack(pred_list2, 0).mean(0))
        pred_mask1.append(torch.stack(pred_mask2, 0).mean(0))
            
preds = torch.cat(pred_list1).numpy()
masks = torch.cat(pred_mask1).numpy()[:,0]

553it [1:16:35,  8.31s/it]


In [11]:
preds.shape

(17679, 5)

In [12]:
masks.shape

(17679, 16, 16)

In [13]:
np.save('/workspace/data/external/pseudo/preds_1', preds)
np.save('/workspace/data/external/pseudo/masks_1', masks)